<span style="color:orange">
 <h2> Databricks-Assignment 1 (Basic Common Transformation)
</span>
  <h5>
    <span style="color:red">
<b>Author: Deepak Goyal <br>
   <a> Assignment completed by : Roopmathi Gunna </a><br>
   written and executed on DBR 12.2 in DB Community edition
</span>

In [0]:
#Load employee data into a Dataframe
input = "/FileStore/Azurelib files/EmployeeData_input.csv"
df = spark.read.csv(input,header = True, inferSchema = True)
df.printSchema()
df.show(5)
from pyspark.sql.functions import *


root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- creationDate: timestamp (nullable = true)
 |-- bonus: double (nullable = true)

+---+----------+---------+--------------------+------+------+-------------------+--------+
| id|first_name|last_name|               email|gender|salary|       creationDate|   bonus|
+---+----------+---------+--------------------+------+------+-------------------+--------+
|  1|    Valene|   Ingley|vingley0@livejour...|Female| 44104|1957-09-09 16:44:40|    null|
|  2|  Lynnelle|    Hurll| lhurll1@answers.com|Female|112411|1907-05-10 17:38:56|    null|
|  3|   Miranda|    Train|   mtrain2@imgur.com|Female| 91073|1941-01-24 16:05:23|12875.54|
|  4|    Dulsea|     Foss|dfoss3@dagondesig...|Female|193291|1942-05-09 20:59:39|    null|
|  5|    Anatol|  Dunklee| adunklee4@goog

<b> Q1. How many employees are there in the input file?

In [0]:
# Write your code using the Spark Function
tot_employees = df.count()
print (tot_employees) 

#another approach 
from pyspark.sql.functions import count
df.select(count("*").alias("Total_employees")).show()

# seems we have some rows that have null value in id column 

+---------------+
|Total_employees|
+---------------+
|           1000|
+---------------+



<b> Q2. What is the average salary of all the employees?

In [0]:
# Write your code using the Spark Function

avg_salary = df.select(round(avg("salary"), 2).alias("Average_Salary"))
avg_salary.show()

+--------------+
|Average_Salary|
+--------------+
|     100011.38|
+--------------+



<b> Q3. How many male and female employees are there in the input file?

In [0]:
# Write your code using the Spark Function
male_female_count = df.groupBy("Gender").agg(count("*").alias("Count")).filter(col("Gender").isNotNull())
male_female_count.show()

# For an interactive bar viz I decided to also use diplay
display(male_female_count)
# There are 28 rows with a null value for Gender column, thats why I have used filter.isNotNull()

+------+-----+
|Gender|Count|
+------+-----+
|Female|  501|
|  Male|  471|
+------+-----+



Gender,Count
Female,501
Male,471


Databricks visualization. Run in Databricks to view.

<b> Q4. Who is the employee with the highest salary?

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col

# Find the employee with the highest salary
highest_salary_employee = df.orderBy(col("salary").desc()).limit(1).select(
    "first_name",
    "last_name",
    "salary"
).first()

# Prints the result
print(f"The employee with the highest salary is {highest_salary_employee['first_name']} {highest_salary_employee['last_name']} with a salary of {highest_salary_employee['salary']}")




The employee with the highest salary is Alfonso Hanbridge with a salary of 199846


<b> Q5. What is the total bonus amount paid to all the employees?

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import sum
df.select(round(sum("Bonus"), 2).alias("Total_Bonus_Paid")).display()
 

Total_Bonus_Paid
4.25048123E7



<b> Q6. How many employees were hired in the year 2020?

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import year

df_2020 = df.filter(year("creationDate") == 2020)
total_employees_2020 = df_2020.count()
print("Number of employees hired in 2020:", total_employees_2020)


Number of employees hired in 2020: 7


 
<b> Q7. Who are the top 5 employees with the highest salaries?

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import concat, col, desc
top_5_salaries = df.orderBy(desc("salary")).limit(5).select(concat(col("first_name"), lit(" "), col("last_name")).alias("Employee Name"), "Salary")
top_5_salaries.show(truncate=False)

#another approach

top_5_salaries2 = df.select(concat(col("first_name"),lit(" "),col("last_name")).alias("Employee Name"), "salary").orderBy(desc("salary")).limit(5)
top_5_salaries2.show() 

+-----------------+------+
|Employee Name    |Salary|
+-----------------+------+
|Alfonso Hanbridge|199846|
|Dermot Letchford |199484|
|Jennee Ballingal |198873|
|Griswold Kocher  |198688|
|Baryram Whitham  |198672|
+-----------------+------+

+-----------------+------+
|    Employee Name|salary|
+-----------------+------+
|Alfonso Hanbridge|199846|
| Dermot Letchford|199484|
| Jennee Ballingal|198873|
|  Griswold Kocher|198688|
|  Baryram Whitham|198672|
+-----------------+------+



<b> Q8. What is the total salary paid to all male employees?

In [0]:
# Write your code using the Spark Function

total_salary_male = df.filter(df.gender == 'Male').agg(sum("salary").alias("total_salary_male")).collect()[0]["total_salary_male"]
print("Total salary paid to male employees:", total_salary_male)

#another approach using first()
total_salary_male = df.filter(df.gender == 'Male').agg(sum("salary").alias("total_salary_male")).first()["total_salary_male"]
print("Total salary paid to male employees:", total_salary_male)



Total salary paid to male employees: 44253777
Total salary paid to male employees: 44253777



<b> Q9. Who is the employee with the highest bonus amount?

In [0]:
# Write your code using the Spark Function# Write your code using the Spark Function
highest_bonus_employee = (df.orderBy(desc("bonus"))
                          .limit(1)
                          .select(concat(col("first_name"), lit(" "), col("last_name")).alias("Employee Name"), "bonus"))

highest_bonus_employee.show()



+---------------+--------+
|  Employee Name|   bonus|
+---------------+--------+
|Franni Chessell|99993.16|
+---------------+--------+




<b> Q10. What is the earliest creation date of any employee in the input file?

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import min

earliest_creation_date = df.agg(min("creationDate").cast("date").alias("earliest_creation_date")).collect()[0]["earliest_creation_date"]
print("Earliest creation date of any employee:", (earliest_creation_date))

Earliest creation date of any employee: 1900-07-08



<b> Q11. How many employees have a salary greater than 100,000?

In [0]:
# Write your code using the Spark Function

high_salary_count = df.filter(df.salary > 100000).count()
print("Number of employees with salary greater than 100,000:", high_salary_count)


Number of employees with salary greater than 100,000: 480



<b> Q12.  Who is the female employee with the highest salary?

In [0]:
# Write your code using the Spark Function

highest_salary_female = df.filter(df.gender == 'Female').orderBy(desc("salary")).limit(1).select("first_name","last_name","salary")
highest_salary_female.show()


+----------+---------+------+
|first_name|last_name|salary|
+----------+---------+------+
|    Jennee|Ballingal|198873|
+----------+---------+------+




<b> Q13. What is the total bonus amount paid to female employees?

In [0]:
# Write your code using the Spark Function

total_bonus_female = df.filter(df.gender == 'Female').agg(sum("bonus").alias("total_bonus_female")).collect()[0]["total_bonus_female"]
print("Total bonus amount paid to female employees:", total_bonus_female)


Total bonus amount paid to female employees: 21323153.59



<b> Q14. How many employees have a salary between 50,000 and 75,000?

In [0]:
# Write your code using the Spark Function

salary_range_count = df.filter((df.salary >= 50000) & (df.salary <= 75000)).count()
print("Number of employees with salary between 50,000 and 75,000:", salary_range_count)
# This approach is faster 0.53 seconds execution time 

#another approach using between()

salary_range_count = df.filter(col("salary").between(50000, 75000)).count()
print("Number of employees with salary between 50,000 and 75,000:", salary_range_count)
# This took 0.69 seconds



Number of employees with salary between 50,000 and 75,000: 117



<b> Q15. Who are the top 10 employees with the highest bonuses?

In [0]:
# Write your code using the Spark Function

from pyspark.sql.functions import desc
top_10_bonuses = df.select("first_name", "last_name", "bonus").orderBy(desc("bonus")).limit(10)
top_10_bonuses.display()

first_name,last_name,bonus
Franni,Chessell,99993.16
Asher,Dupoy,99985.66
Lovell,Ingall,99932.43
Sullivan,Gyenes,99845.6
Jessie,Grgic,99650.89
Nicol,Bruton,99581.27
Nonah,Tompkiss,99046.27
Andy,Morch,99033.16
Swen,Hinrichsen,98721.92
Dorotea,Kynan,98639.27


Databricks visualization. Run in Databricks to view.


<b> Q16. What is the latest creation date of any employee in the input file?

In [0]:
# Write your code using the Spark Function

latest_creation_date = df.agg(max("creationdate").cast("date").alias("latest_creation_date")).collect()[0]["latest_creation_date"]
print("Latest creation date of any employee:", latest_creation_date)


Latest creation date of any employee: 2023-03-24



<b> Q17. How many employees have a salary less than 50,000?

In [0]:
# Write your code using the Spark Function

low_salary_count = df.filter(df.salary < 50000).count()
print("Number of employees with salary less than 50,000:", low_salary_count)


Number of employees with salary less than 50,000: 234



<b> Q18. Who are the top 3 employees with the lowest salaries?

In [0]:
# Write your code using the Spark Function

top_3_lowest_salaries = df.filter(col("salary").isNotNull()).orderBy("salary").limit(3)
top_3_lowest_salaries.show()

+---+----------+----------+--------------------+------+------+-------------------+--------+
| id|first_name| last_name|               email|gender|salary|       creationDate|   bonus|
+---+----------+----------+--------------------+------+------+-------------------+--------+
|809|  Hercules|Pollington|hpollingtonmg@chi...|  Male|   109|1936-08-10 16:43:14| 57493.2|
|381|     Olwen| O'Scanlon| ooscanlonak@usa.gov|Female|   184|2017-10-11 06:21:17|22415.01|
|542| Heriberto|      null| hsurmeyersf1@un.org|  Male|   479|               null|51871.41|
+---+----------+----------+--------------------+------+------+-------------------+--------+




<b> Q19. What is the average bonus amount paid to all employees?

In [0]:
# Write your code using the Spark Function
# Assuming there is a 'bonus' column in your DataFrame
average_bonus = df.agg(avg("bonus").alias("average_bonus")).collect()[0]["average_bonus"]
print("Average bonus amount paid to all employees:", average_bonus)


Average bonus amount paid to all employees: 50965.00275779381



<b> Q20. How many employees have a bonus greater than their salary?

In [0]:
# Write your code using the Spark Function
# Assuming there are 'bonus' and 'salary' columns in your DataFrame
bonus_greater_than_salary_count = df.filter(df.bonus > df.salary).count()
print("Number of employees with a bonus greater than their salary:", bonus_greater_than_salary_count)


Number of employees with a bonus greater than their salary: 194
